## EXERCÍCIO - RDD com Partições

In [1]:
spark

In [2]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [3]:
# listando os arquivos que estão dentro do container jupyter-spark
!ls /opt/spark/logs

entrada1.txt
entrada2.txt
spark--org.apache.spark.deploy.master.Master-1-jupyter-notebook.out


In [4]:
# listando o conteúdo do arquivo abaixo

!cat /opt/spark/logs/spark--org.apache.spark.deploy.master.Master-1-jupyter-notebook.out

# arquivo de texto com logs do spark

Spark Command: /opt/java/bin/java -cp /etc/spark/:/opt/spark/jars/*:/etc/hadoop/:/etc/hadoop/*:/opt/hadoop/share/hadoop/common/lib/*:/opt/hadoop/share/hadoop/common/*:/opt/hadoop/share/hadoop/hdfs/*:/opt/hadoop/share/hadoop/hdfs/lib/*:/opt/hadoop/share/hadoop/yarn/lib/*:/opt/hadoop/share/hadoop/yarn/*:/opt/hadoop/share/hadoop/mapreduce/lib/*:/opt/hadoop/share/hadoop/mapreduce/*:/opt/hadoop/share/hadoop/tools/lib/* -Xmx1g org.apache.spark.deploy.master.Master --host localhost --port 7077 --webui-port 8080
20/03/18 20:31:09 INFO master.Master: Started daemon with process name: 1087@jupyter-notebook
20/03/18 20:31:09 INFO util.SignalUtils: Registered signal handler for TERM
20/03/18 20:31:09 INFO util.SignalUtils: Registered signal handler for HUP
20/03/18 20:31:09 INFO util.SignalUtils: Registered signal handler for INT
20/03/18 20:31:10 INFO spark.SecurityManager: Changing view acls to: root
20/03/18 20:31:10 INFO spark.SecurityManager: Changing modify acls to: root
20/03/18 20:31:10 IN

## 1. Ler com RDD os arquivos localmente do diretório “/opt/spark/logs/” ("file:///opt/spark/logs/") com 10 partições

### Todas as ações terão 10 partições, por o RDD foi criado com 10 partições

In [5]:
# Criado o RDD
logs_particoes = sc.textFile("file:///opt/spark/logs/spark--org.apache.spark.deploy.master.Master-1-jupyter-notebook.out")

In [6]:
# por padrão cria com 2 partições
logs_particoes.getNumPartitions()

2

In [7]:
# Criado o RDD com 10 Partições

logs_particoes = sc.textFile("file:///opt/spark/logs/spark--org.apache.spark.deploy.master.Master-1-jupyter-notebook.out",10)

In [8]:
# Criado com 10 partições
logs_particoes.getNumPartitions()

10

In [9]:
# separando as palavras
palavras_particoes = logs_particoes.flatMap(lambda linha: linha.split(" "))

In [14]:
# criou com 10 partições

palavras_particoes.getNumPartitions()

10

In [10]:
# separou as palavras em linhas dentro de uma lista

# visualizando as 6 primeiras posições de memória

palavras_particoes.take(6)

['Spark',
 'Command:',
 '/opt/java/bin/java',
 '-cp',
 '/etc/spark/:/opt/spark/jars/*:/etc/hadoop/:/etc/hadoop/*:/opt/hadoop/share/hadoop/common/lib/*:/opt/hadoop/share/hadoop/common/*:/opt/hadoop/share/hadoop/hdfs/*:/opt/hadoop/share/hadoop/hdfs/lib/*:/opt/hadoop/share/hadoop/yarn/lib/*:/opt/hadoop/share/hadoop/yarn/*:/opt/hadoop/share/hadoop/mapreduce/lib/*:/opt/hadoop/share/hadoop/mapreduce/*:/opt/hadoop/share/hadoop/tools/lib/*',
 '-Xmx1g']

In [11]:
# depois de modificado com o flatMap, utilizar o Map para modificar as palavras

palavras_minusculas_particoes = palavras_particoes.map(lambda palavra: palavra.lower())

In [15]:
# criou com 10 partições
palavras_minusculas_particoes.getNumPartitions()

10

In [12]:
# as palavras nas linhas estão em minúsculo

palavras_minusculas_particoes.take(5)

['spark',
 'command:',
 '/opt/java/bin/java',
 '-cp',
 '/etc/spark/:/opt/spark/jars/*:/etc/hadoop/:/etc/hadoop/*:/opt/hadoop/share/hadoop/common/lib/*:/opt/hadoop/share/hadoop/common/*:/opt/hadoop/share/hadoop/hdfs/*:/opt/hadoop/share/hadoop/hdfs/lib/*:/opt/hadoop/share/hadoop/yarn/lib/*:/opt/hadoop/share/hadoop/yarn/*:/opt/hadoop/share/hadoop/mapreduce/lib/*:/opt/hadoop/share/hadoop/mapreduce/*:/opt/hadoop/share/hadoop/tools/lib/*']

In [18]:
# contando a quantidade de palavras em ordem descrescente
palavras_count_particoes = palavras_minusculas_particoes.map(lambda palavra: (palavra,1))

In [19]:
palavras_count_particoes.getNumPartitions()

10

In [20]:
# Palavras em minúscilas e contando a quantidade
palavras_count_particoes.take(10)

[('spark', 1),
 ('command:', 1),
 ('/opt/java/bin/java', 1),
 ('-cp', 1),
 ('/etc/spark/:/opt/spark/jars/*:/etc/hadoop/:/etc/hadoop/*:/opt/hadoop/share/hadoop/common/lib/*:/opt/hadoop/share/hadoop/common/*:/opt/hadoop/share/hadoop/hdfs/*:/opt/hadoop/share/hadoop/hdfs/lib/*:/opt/hadoop/share/hadoop/yarn/lib/*:/opt/hadoop/share/hadoop/yarn/*:/opt/hadoop/share/hadoop/mapreduce/lib/*:/opt/hadoop/share/hadoop/mapreduce/*:/opt/hadoop/share/hadoop/tools/lib/*',
  1),
 ('-xmx1g', 1),
 ('org.apache.spark.deploy.master.master', 1),
 ('--host', 1),
 ('localhost', 1),
 ('--port', 1)]

### Usando o ReduceByKey pode ser atribuído um número específico de partições
### 2. Contar a quantidade de cada palavras em ordem decrescente do RDD em 5 partições

In [21]:
palavras_reduce_particoes = palavras_count_particoes.reduceByKey(lambda chave1, chave2: chave1 + chave2)

In [22]:
# Continua tendo 10 partições
palavras_reduce_particoes.getNumPartitions()

10

In [23]:
# CRIANDO COM 5 PARTIÇÕES
palavras_reduce_particoes = palavras_count_particoes.reduceByKey(lambda chave1, chave2: chave1 + chave2, 5)

In [24]:
# CRIADO tendo 5 partições
palavras_reduce_particoes.getNumPartitions()

5

In [25]:
palavras_reduce_particoes.take(10)

[('--port', 1),
 ('20:31:09', 4),
 ('daemon', 1),
 ('process', 1),
 ('registered', 3),
 ('signal', 3),
 ('20:31:10', 24),
 ('to:', 4),
 ('groups', 4),
 ('', 4)]

In [26]:
palavras_reduce_particoes.count()

107

### 3. Salvar o RDD no diretório do HDFS /user/<seu-nome>/logs_count_word_5

In [27]:
palavras_reduce_particoes.saveAsTextFile("/user/feliciani/logs_count_word_5")

In [28]:
!hdfs dfs -ls /user/feliciani/logs_count_word_5

# CRIADO COM AS 5 PARTIÇÕES

Found 6 items
-rw-r--r--   2 root supergroup          0 2021-06-28 18:35 /user/feliciani/logs_count_word_5/_SUCCESS
-rw-r--r--   2 root supergroup        575 2021-06-28 18:35 /user/feliciani/logs_count_word_5/part-00000
-rw-r--r--   2 root supergroup        208 2021-06-28 18:35 /user/feliciani/logs_count_word_5/part-00001
-rw-r--r--   2 root supergroup        444 2021-06-28 18:35 /user/feliciani/logs_count_word_5/part-00002
-rw-r--r--   2 root supergroup        772 2021-06-28 18:35 /user/feliciani/logs_count_word_5/part-00003
-rw-r--r--   2 root supergroup        785 2021-06-28 18:35 /user/feliciani/logs_count_word_5/part-00004


### 4. Refazer a questão 2, com todas as funções na mesma linha de um RDD

In [31]:
logs_particoes.count()

30

In [36]:
# Todas as funções na mesma linha

palavras_particoes_linha = logs_particoes.flatMap(lambda linha: linha.split(" ")).map(lambda palavra: palavra.lower()).map(lambda palavra: (palavra,1)).reduceByKey(lambda chave1, chave2: chave1 + chave2)

In [37]:
palavras_particoes_linha.count()

107